In [1]:
import tensorflow as tf
from flows import NormalRW, DFlow, NVPFlow, LogNormal, GVAR, phase,\
Normal, floatX, MVNormal, MVNormalRW, Linear, LinearChol
import flows

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from tensorflow.contrib.distributions import WishartCholesky
import math

np.random.seed(1234)
tf.set_random_seed(1234)

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
!ls data

test_aus_data.csv  test_rus_data.csv


In [3]:
datas = ['./data/test_rus_data.csv', './data/test_aus_data.csv']

In [4]:
datas = [pd.read_csv(x).values.astype(floatX).T[np.newaxis][:,1:]/100 for x in datas]

In [5]:
scaler = 0.
for i, data in enumerate(datas):
    stds = (data[0,1:] - data[0,:-1]).std(axis=0)
    print(stds)
    scaler = scaler + stds
    datas[i] = data
print('---')
scaler /= len(datas)
for i in range(len(datas)):
    datas[i] /= scaler
    data = datas[i]
    stds = (data[0,1:] - data[0,:-1]).std(axis=0)
    print(stds)
    data = np.concatenate([data[:,1:], data[:,:-1]], axis=-1)
    datas[i] = data

[0.38600643 0.16409765 0.01116647]
[0.0186606  0.01934265 0.00918617]
---
[1.90777308 1.78911236 1.0972991 ]
[0.09222692 0.21088764 0.9027009 ]


In [6]:
country_data = {'rus':datas[0], 'aus':datas[1]}

In [7]:
class VARmodel:
    def __init__(self, data, name='VARmodel', mu=None):
        self.data = data
        self.NUM_STEPS = data.shape[1]
        self.name = name
        self.logdensities = []
        self.priors = []
        self.dim = [3,3*2+1]
        self.summaries = []
        
        self.stop_time = tf.placeholder_with_default(self.NUM_STEPS, ())
        self.observable_mask = tf.range(0, data.shape[1], dtype=tf.int32) < self.stop_time
        
        pd = np.mean(np.std(data[0,1:] - data[0,:-1], axis=0))
#         print(pd)
        
        with tf.variable_scope(name) as scope:
            self.scope = scope
            
            self.create_rw_priors()
            self.outputs = self.create_walk_inference(mu=mu)
            self.create_observ_dispersion_inference(pd*0.5)
            self.create_likelihood(self.observable_mask, self.outputs)
            self.summary = tf.summary.merge(self.summaries)
            
    def create_summary(self, stype, name, tensor):
        s = stype(name, tensor)
        self.summaries.append(s)

    def create_rw_priors(self):
        dim = self.dim
        with tf.variable_scope('rw_priors'):
            with tf.variable_scope('walk_ord'):
                s1_prior_d = LogNormal(1, mu=math.log(0.01), sigma=6., name='s1_prior')

                with tf.variable_scope('s1_inference', dtype=floatX):
                    mu = tf.get_variable('mu', shape=[1], 
                                         initializer=tf.constant_initializer(s1_prior_d.mu))

                    logsigma_init = tf.constant_initializer(min(math.log(s1_prior_d.sigma), -1.))
                    logsigma = tf.get_variable('logsigma', shape=[1], 
                                               initializer=logsigma_init)
                    sigma = tf.exp(logsigma)
                    s1_d = LogNormal(1, mu=mu, sigma=sigma)

                s1 = s1_d.sample()
                
                self.create_summary(tf.summary.scalar, 's1_ord', s1[0])
                
                self.logdensities.append(s1_d.logdens(s1))

                s1_prior = s1_prior_d.logdens(s1)
                self.priors.append(s1_prior)

            s0 = 80.

            K = dim[0] * dim[1]

            PWalk = NormalRW(dim=None, sigma0=None, mu0=None, sigma=s1, name='OrdWalk')
            self.PWalk = PWalk
                
    def create_walk_inference(self, mu=None):
        dim = self.dim
        gvar = GVAR(dim=dim[0]*dim[1], len=self.NUM_STEPS, name='coef_rw_inference', mu=mu)
        outputs = gvar.sample()
        
        self.logdensities.append(gvar.logdens)
        self.priors.append(self.PWalk.logdens(outputs, reduce=True))
        
        return outputs
    
    def create_observ_dispersion_inference(self, prior_disp):
        with tf.variable_scope('obs_d_inf', reuse=tf.AUTO_REUSE):
            ldiag = DFlow([NVPFlow(dim=3, name='ldiag_flow_' + str(i)) for i in range(2)])
            ldiag.logdens -= tf.reduce_sum(ldiag.output)
        
        self.obs_d = MVNormal(3, sigma=None, name='obs_d_prior', 
                   ldiag=ldiag.output[0]-math.log(0.5*prior_disp))
        
        df = 3
        pmat = np.diag([(1/(0.5*prior_disp)**2)]*3)/df
        cov_prior = WishartCholesky(df, pmat, cholesky_input_output_matrices=True)
        
        pr = cov_prior.log_prob(self.obs_d.fsigma)
        self.logdensities.append(ldiag.logdens[0])
        self.priors.append(pr)
        self.create_summary(tf.summary.histogram, 'obs_disp', tf.sqrt(tf.diag(self.obs_d.sigma)))
#         self.create_summary(tf.summary.histogram, 'obs_cov_diag', tf.sqrt(1/tf.exp(ldiag.output[0])))
#         obs_d_prior = LogNormal(dim=None, mu=math.log(prior_disp), sigma=0.5)

#         with tf.variable_scope('obs_cov_inference', dtype=floatX):
#             mu = tf.get_variable('mu', shape=[1], initializer=tf.constant_initializer(obs_d_prior.mu))
#             logsigma = tf.get_variable('logsigma', shape=[1], initializer=tf.constant_initializer(-5))
#             sigma = tf.exp(logsigma)

#             obs_d_post = LogNormal(1, mu=mu, sigma=sigma)
#             obs_ds = obs_d_post.sample()
#             self.obs_ds = obs_ds

#             obs_ds_logdens = obs_d_post.logdens(obs_ds)
            
#             self.logdensities.append(obs_ds_logdens)
#             self.priors.append(obs_d_prior.logdens(obs_ds))

#             self.create_summary(tf.summary.scalar, 'observ_sigma', obs_ds[0])
            
    def predict(self, observable_mask, outputs):
        dim = self.dim
        data = self.data
        out = tf.reshape(outputs, [self.NUM_STEPS, dim[0], dim[1]])

        def step(prev, x):
            mask = x[0]
            prev_pred = tf.where(mask, x[1], prev)[tf.newaxis]
            params = x[2]

            d0 = params[:,:dim[0]]
            d1 = params[:,dim[0]:2*dim[0]]

            pp1 = prev_pred[:,:dim[0]]
            pp0 = prev_pred[:,dim[0]:2*dim[0]]

            new_pred = tf.matmul(pp0, d0)[0] + tf.matmul(pp1, d1)[0]+ params[:,-1] + pp1[0]
            new_pred = tf.concat([new_pred, pp1[0]], axis=0)
            return new_pred

        ar = tf.scan(step, [observable_mask, data[0], out], initializer=tf.zeros([2*dim[0]], dtype=floatX))
        return ar
    
    def create_likelihood(self, observable_mask, outputs):
        dim = self.dim
        obs_d = self.obs_d#Normal(dim=None, sigma=self.obs_ds, mu=0)
        
        preds = self.predict(observable_mask, outputs)
        
        diffs = preds[:-1] - data[0,1:]
        diffs = diffs[:,:dim[0]]

        logl = obs_d.logdens(diffs, reduce=False)
        print(logl)
        logl *= tf.cast(self.observable_mask[:-1], floatX)[:,tf.newaxis]

        logl = tf.reduce_sum(logl)
        self.create_summary(tf.summary.scalar, 'loglikelihood', logl)
        self.priors.append(logl)

In [8]:
global_inf = DFlow([NVPFlow(dim=(3*2+1)*3, name='flow_{}'.format(i)) for i in range(4)], init_sigma=0.08)
global_prior = Normal(None, sigma=10.).logdens(global_inf.output)

In [9]:
global_inf.logdens

<tf.Tensor 'sub:0' shape=(1,) dtype=float64>

In [10]:
models = []
indiv_logldens = []
indiv_priors = []
indivs = {}
individ_variation_prior = Normal((3*2+1)*3, sigma=3., mu=global_inf.output[0])


for country, data in country_data.items():
    with tf.variable_scope(country):
        individ_variation = DFlow([LinearChol((3*2+1)*3, name='lc')], init_sigma=0.08)
        ind = individ_variation.output[0]
        indivs[country] = ind
        
        indiv_logldens.append(individ_variation.logdens)
        indiv_priors.append(individ_variation_prior.logdens(ind))
    
    model = VARmodel(data, name='{}_model'.format(country), mu=ind[tf.newaxis])
    models.append(model)

rus_model/obs_d_prior
Tensor("rus_model/obs_d_prior_2/sub_1:0", shape=(19,), dtype=float64)
aus_model/obs_d_prior
Tensor("aus_model/obs_d_prior_2/sub_1:0", shape=(35,), dtype=float64)


In [11]:
prior = tf.reduce_sum([model.priors for model in models])\
+ tf.reduce_sum(indiv_priors) + global_prior

logdensity = tf.reduce_sum([model.logdensities for model in models])\
+ tf.reduce_sum(indiv_logldens) + global_inf.logdens[0]

In [12]:
kl = logdensity - prior

In [13]:
kls = tf.summary.scalar('KLd', kl)
summary = tf.summary.merge([kls] + [model.summaries for model in models])

In [14]:
main_op = tf.train.AdamOptimizer(0.0001).minimize(kl)

In [15]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()

In [16]:
init.run()

In [17]:
# !rm -R /tmp/hier

In [18]:
writer = tf.summary.FileWriter('/tmp/hier/aus_rus_hier_fullcovobs')

In [19]:
epoch = 0

In [20]:
for epoch in range(epoch, 100000):
    for step in range(100):
        sess.run(main_op)
    s, _ = sess.run([summary, main_op])
    writer.add_summary(s, global_step=epoch)

InvalidArgumentError: Nan in summary histogram for: rus_model/obs_disp
	 [[Node: rus_model/obs_disp = HistogramSummary[T=DT_DOUBLE, _device="/job:localhost/replica:0/task:0/device:CPU:0"](rus_model/obs_disp/tag, rus_model/Sqrt)]]

Caused by op 'rus_model/obs_disp', defined at:
  File "/usr/lib64/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib64/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/nikita/.virtualenvs/py3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/nikita/.virtualenvs/py3.6/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/nikita/.virtualenvs/py3.6/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/nikita/.virtualenvs/py3.6/lib/python3.6/site-packages/tornado/ioloop.py", line 888, in start

In [ ]:
global_post=global_inf.output[0]

In [ ]:
ss = []
for _ in range(1000):
    ss.append(global_post.eval())
ss = np.array(ss)

In [ ]:
np.mean(ss,axis=0)

In [ ]:
ss.std(axis=0)

In [ ]:
sns.kdeplot(ss[:,2], ss[:,1])
plt.show()

In [ ]:
ind

In [21]:
for data in datas:
    print(np.max(np.std(data[0,1:] - data[0,:-1], axis=0)))

1.9475575432574979
0.9155053293865658
